# Test pgVector Search Function

This notebook tests the new PostgreSQL pgvector search functionality after migration from Chroma.

In [ ]:
# Install required packages if not already installed
# !uv add asyncpg pandas numpy python-dotenv langchain-community

In [ ]:
import asyncpg
from dotenv import load_dotenv
from langchain_community.embeddings import DeepInfraEmbeddings

# Load environment variables
load_dotenv()
import os

# Configuration
POSTGRES_CONFIG = {
    "host": os.getenv("POSTGRES_HOST", "localhost"),
    "port": int(os.getenv("POSTGRES_PORT", "5432")),
    "database": os.getenv("POSTGRES_DB", "vector_search"),
    "user": os.getenv("POSTGRES_USER", "postgres"),
    "password": os.getenv("POSTGRES_PASSWORD", "postgres"),
}

print("Configuration loaded:")
print(
    f"PostgreSQL: {POSTGRES_CONFIG['host']}:{POSTGRES_CONFIG['port']}/{POSTGRES_CONFIG['database']}"
)

Configuration loaded:
PostgreSQL: localhost:5432/vector_search


In [2]:
# Initialize embedding model
embeddings = DeepInfraEmbeddings(
    model_id="BAAI/bge-m3",
    query_instruction="",
    embed_instruction="",
    deepinfra_api_token=os.getenv("DEEP_INFRA_API_KEY"),
)

print("Embedding model initialized")

Embedding model initialized


## 1. Test Database Connection

In [3]:
async def test_connection():
    """Test PostgreSQL connection"""
    conn = await asyncpg.connect(**POSTGRES_CONFIG)

    # Test basic connection
    version = await conn.fetchval("SELECT version()")
    print(f"Connected to PostgreSQL: {version[:50]}...")

    # Check if pgvector extension is enabled
    vector_version = await conn.fetchval(
        "SELECT extversion FROM pg_extension WHERE extname = 'vector'"
    )
    print(f"pgvector extension version: {vector_version}")

    # Check tables
    tables = await conn.fetch(
        "SELECT table_name FROM information_schema.tables WHERE table_schema = 'public'"
    )
    print(f"Tables in database: {[t['table_name'] for t in tables]}")

    # Check document count
    doc_count = await conn.fetchval("SELECT COUNT(*) FROM document_embeddings")
    file_count = await conn.fetchval("SELECT COUNT(*) FROM files")
    print(f"Documents in database: {doc_count}")
    print(f"Files in database: {file_count}")

    await conn.close()


await test_connection()

Connected to PostgreSQL: PostgreSQL 16.11 (Debian 16.11-1.pgdg12+1) on aarc...
pgvector extension version: 0.8.1
Tables in database: ['files', 'document_embeddings']
Documents in database: 239
Files in database: 13


## 2. Test Vector Search Function

In [48]:
# Test the search_vectors function from the router
import importlib
import sys

sys.path.append("..")

import app.routers.vector_search as vector_search_module

importlib.reload(vector_search_module)

from app.routers.vector_search import (
    VectorSearchRequest,
    get_embeddings_model,
    search_vectors,
)

In [49]:
# Create a mock request object for get_db_pool dependency
class MockRequest:
    class MockState:
        def __init__(self, db_pool):
            self.db_pool = db_pool

    def __init__(self, db_pool):
        self.app = type("app", (), {"state": self.MockState(db_pool)})()


# Create database connection
conn = await asyncpg.connect(**POSTGRES_CONFIG)

# Create mock request with the connection pool
mock_request = MockRequest(conn)

# Get the embeddings model
embeddings_model = await get_embeddings_model()

In [50]:
# Test vector search with time decay (freshness boosting)
db_pool = await asyncpg.create_pool(**POSTGRES_CONFIG, min_size=1, max_size=10)

request_time_decay = VectorSearchRequest(
    query="CIO view on Global Equity",
    limit=5,
    threshold=0.45,
    filter_metadata=None,
    use_time_decay=True,
    time_decay_grace_days=7,
    time_decay_cutoff_days=180,
)

try:
    results_time_decay = await search_vectors(
        request_data=request_time_decay,
        db_pool=db_pool,
        embeddings_model=embeddings_model,
    )
    print(f"Found {len(results_time_decay)} results (time decay enabled)")
    if results_time_decay:
        top = results_time_decay[0]
        print(f"Top result similarity (after decay): {top.similarity:.4f}")
        print(f"Top file: {top.file_name}")
except Exception as e:
    print(f"Error: {e}")
    import traceback

    traceback.print_exc()
finally:
    await db_pool.close()

Found 5 results (time decay enabled)
Top result similarity (after decay): 0.5183
Top file: KKP_Mandate_Model_Portfolios_Performance_Mid Year 2025_with_Holdings&Rebalance.pdf


In [47]:
# Create a connection pool (not just a connection)
db_pool = await asyncpg.create_pool(**POSTGRES_CONFIG, min_size=1, max_size=10)

# Create search request
request = VectorSearchRequest(
    query="CIO View on Global Equity",
    limit=5,
    threshold=0.5,
    filter_metadata={"file_md5_checksum": "c9cf965704af5223f3bacf56301b3de2"},
)

# Call the actual search_vectors function from the router
try:
    # Now we pass the actual pool which has acquire() method
    result = await search_vectors(
        request_data=request, db_pool=db_pool, embeddings_model=embeddings_model
    )

    print(f"Found {len(result)} results:")
    for i, item in enumerate(result, 1):
        print(f"\n{i}. Chunk ID: {item.chunk_id}")
        print(f"   Similarity: {item.similarity:.4f}")
        print(f"   File: {item.file_name}")
        print(f"   Page: {item.page_number}")
        print(f"   Type: {item.chunk_type}")
        print(f"   Content preview: {item.content[:200]}...")
        print("-" * 50)

except Exception as e:
    print(f"Error: {e}")
    import traceback

    traceback.print_exc()

finally:
    await db_pool.close()

Found 5 results:

1. Chunk ID: 20251124_weekly_kkps_advisory_page_1
   Similarity: 0.5590
   File: 20251124_Weekly_KKPS Advisory.pdf
   Page: 1
   Type: page
   Content preview: KKPS ADVISORY | NOVEMBER 2025

CIO VIEW

มุมมองตลาดหุ้นปรับขึ้นดีในปี 2026 จากกำไรที่เติบโตแข็งแรง แต่จะเผชิญความผันผวนจากมุมมองค่ำงลด้าน Valuation

ตัวเลขการจ้างงานสหรัฐฯ ยังแข็งแกร่ง ในขณะที่เงินเฟ้...
--------------------------------------------------

2. Chunk ID: 20251124_weekly_kkps_advisory_page_26
   Similarity: 0.5414
   File: 20251124_Weekly_KKPS Advisory.pdf
   Page: 26
   Type: page
   Content preview: logo: Equity Solutions with tagline "Fuel Your Portfolio"
Company Name & Tagline :
  • The logo displays the company name "Equity Solutions".
  • Tagline beneath the name reads "Fuel Your Portfolio".
...
--------------------------------------------------

3. Chunk ID: 20251124_weekly_kkps_advisory_page_27
   Similarity: 0.5228
   File: 20251124_Weekly_KKPS Advisory.pdf
   Page: 27
   Type: page
   Con

### Test 1: Basic Search

In [22]:
import time

# Test basic search with timing
start_time = time.time()
results1 = await test_vector_search(
    query="CIO view on Global Equity", limit=5, threshold=0.5
)
end_time = time.time()
print(f"Search completed in {end_time - start_time:.4f} seconds")

Generating embedding for query: 'CIO view on Global Equity'
Embedding dimension: 1024

Executing search with threshold=0.5, limit=5

Found 5 results:
--------------------------------------------------------------------------------

1. Chunk ID: 20250729_weekly_kkps_advisory_page_1
   Similarity: 0.5762
   File: 20250729_Weekly_KKPS Advisory.pdf
   Page: 1
   Type: page
   Content preview: KKPS ADVISORY | JULY 2025

- CIO VIEW
- ตลาดยังไม่ฟองสบู่ ทยอยสะสมหุ้นโลกสำหรับผลตอบแทนปีหน้า
  - • หุ้นโลก: ทยอยสะสม DCA เพื่อเฉลี่ยต้นทุน แม้ปัจจุบันจะ all time high มองปัจจัยบวกชัดเจนปีหน้า
    - →...

2. Chunk ID: 20251124_weekly_kkps_advisory_page_1
   Similarity: 0.5424
   File: 20251124_Weekly_KKPS Advisory.pdf
   Page: 1
   Type: page
   Content preview: KKPS ADVISORY | NOVEMBER 2025

CIO VIEW

มุมมองตลาดหุ้นปรับขึ้นดีในปี 2026 จากกำไรที่เติบโตแข็งแรง แต่จะเผชิญความผันผวนจากมุมมองค่ำงลด้าน Valuation

ตัวเลขการจ้างงานสหรัฐฯ ยังแข็งแกร่ง ในขณะที่เงินเฟ้...

3. Chunk ID: kkpxgsam_mandate_service_

### Test 2: Search with File Filter

In [17]:
# First, let's see what files are available
async def list_files():
    conn = await asyncpg.connect(**POSTGRES_CONFIG)
    rows = await conn.fetch(
        "SELECT DISTINCT file_name, file_md5_checksum FROM files ORDER BY file_name"
    )
    await conn.close()
    return rows


files = await list_files()
print("Available files:")
for f in files:
    print(f"  - {f['file_name']} (checksum: {f['file_md5_checksum']})")

Available files:
  - 20250729_Letter from CIO_rc.pdf (checksum: 95ac7aaa3b5722b438fec386e43e5766)
  - 20250729_Weekly_KKPS Advisory.pdf (checksum: 9a5822616cb0b67b18064786f8a878f6)
  - 20250820 Interest Rate Announcement.pdf (checksum: db73d1e2f993eac0153b6dfaa0101570)
  - 20251014_Weekly_KKPS Advisory.pdf (checksum: 0ebe09c4cad4018127dd85b7afcc7bf0)
  - 20251124_Weekly_KKPS Advisory.pdf (checksum: c9cf965704af5223f3bacf56301b3de2)
  - 2026_KKP WM Year Ahead_Cleared.pdf (checksum: beb938aa3cfdd89def1a1e4872832357)
  - Checklist for Mandate PF - USD - 20250228.pdf (checksum: 795d84b7a87efd3b2e2f6554dc0d6921)
  - FCDSS Flow Summary - Front 20250217.pdf (checksum: ef72257d651e17fe3e65a289064c0a26)
  - Indicative Saving Plus Note 3_months_20250806.pdf (checksum: 34a728df50e659f8d952cc891cedd81b)
  - KKP_Mandate_Model_Portfolios_Performance_Mid Year 2025_with_Holdings&Rebalance.pdf (checksum: ca01c90a18bcb76d475370c3381fc2de)
  - KKPxGSAM_Mandate Service Pitchbook 2025.06_Final.pdf (checksu

In [19]:
# Test search with file filter
if files:
    results2 = await test_vector_search(
        query="CIO view on Global Equity",
        limit=3,
        threshold=0.4,
        filters={"file_md5_checksum": "c9cf965704af5223f3bacf56301b3de2"},
    )

Generating embedding for query: 'CIO view on Global Equity'
Embedding dimension: 1024

Executing search with threshold=0.4, limit=3
Filters: {'file_md5_checksum': 'c9cf965704af5223f3bacf56301b3de2'}

Found 3 results:
--------------------------------------------------------------------------------

1. Chunk ID: 20251124_weekly_kkps_advisory_page_1
   Similarity: 0.5424
   File: 20251124_Weekly_KKPS Advisory.pdf
   Page: 1
   Type: page
   Content preview: KKPS ADVISORY | NOVEMBER 2025

CIO VIEW

มุมมองตลาดหุ้นปรับขึ้นดีในปี 2026 จากกำไรที่เติบโตแข็งแรง แต่จะเผชิญความผันผวนจากมุมมองค่ำงลด้าน Valuation

ตัวเลขการจ้างงานสหรัฐฯ ยังแข็งแกร่ง ในขณะที่เงินเฟ้อชะลอการปรับลดลง ทำให้โอกาสที่ Fed จะคงดอกเบี้ยในเดือน ธ.ค. เพิ่มขึ้น ตัวเลขการจ้างงานนอกภาคเกษตรเดือน ก.ย. ปรับเพิ่มขึ้น ในขณะที่ Core PCE อยู่ที่ระดับ 2% ปลายใกล้ระดับเป้าหมายของ Fed ในช่วงหลายเดือนที่ผ่านมา

สรุปมุมมองปี 2026: เน้นลงทุนต่อเนื่องพร้อมกระจายการลงทุน มองเศรษฐกิจสหรัฐฯ โต 2.4% เชื่อว...

2. Chunk ID: 20251124_weekly_kkps_ad

### Test 3: Search with Chunk Type Filter

In [9]:
# Test search with chunk type filter
results3 = await test_vector_search(
    query="financial performance",
    limit=5,
    threshold=0.5,
    filters={"chunk_type": "page"},
)

Generating embedding for query: 'financial performance'
Embedding dimension: 1024

Executing search with threshold=0.5, limit=5
Filters: {'chunk_type': 'page'}

Found 5 results:
--------------------------------------------------------------------------------

1. Chunk ID: kkpxgsam_mandate_service_pitchbook_2025_06_final_page_17
   Similarity: 0.5909
   File: KKPxGSAM_Mandate Service Pitchbook 2025.06_Final.pdf
   Page: 17
   Type: page
   Content preview: SAMPLE PORTFOLIO: INCOME STRATEGIES

<table><thead><tr><th rowspan="2">Target Weight</th><th>Asset Class</th><th>Instrument</th><th>KKP Core Income THB</th><th>KKP Enhanced Income THB</th></tr></thead...

2. Chunk ID: 20250729_weekly_kkps_advisory_page_32
   Similarity: 0.5811
   File: 20250729_Weekly_KKPS Advisory.pdf
   Page: 32
   Type: page
   Content preview: PRINCIPAL GLOBAL CLEAN ENERGY (PRINCIPAL GCLEAN-A)
› Master Fund: VanEck Uranium and Nuclear ETF (NLR)

Fee Structure

บริษัทหลักทรัพย์จัดการกองทุน พรินซิเพิล จำกัด         

### Test 4: Multiple Filters

In [10]:
# Test with multiple filters
results4 = await test_vector_search(
    query="risk management",
    limit=3,
    threshold=0.6,
    filters={
        "chunk_type": "page",
        "file_name": files[0]["file_name"] if files else None,
    },
)

Generating embedding for query: 'risk management'
Embedding dimension: 1024

Executing search with threshold=0.6, limit=3
Filters: {'chunk_type': 'page', 'file_name': '20250729_Letter from CIO_rc.pdf'}

Found 0 results:
--------------------------------------------------------------------------------


## 3. Test API Endpoint Directly

In [7]:
import httpx


async def test_api_endpoint():
    """Test the vector search API endpoint"""
    base_url = "http://localhost:8200"  # Adjust if your API runs on different port

    async with httpx.AsyncClient() as client:
        # Test without embedding (should auto-generate)
        payload = {
            "query": "Latest CIO view and investment recommendations from KKPS CIO; summary of top asset allocation, top sector/region picks, recommended funds/stocks, tactical calls, risk view. Provide most recent CIO commentary and recommended actions.",
            "limit": 5,
            "threshold": 0.4,
            # "filter_metadata": {"chunk_type": "page"},
        }

        print("Testing API endpoint without embedding...")
        response = await client.post(f"{base_url}/pgvector/vector-search", json=payload)

        if response.status_code == 200:
            results = response.json()
            print(f"\nAPI returned {len(results)} results:")
            for i, r in enumerate(results, 1):
                print(f"\n{i}. {r['chunk_id']}")
                print(f"   Similarity: {r['similarity']:.4f}")
                print(f"   File: {r['file_name']}")
                print(f"   Content preview: {r['content'][:100]}...")
        else:
            print(f"Error: {response.status_code} - {response.text}")

In [10]:
await test_api_endpoint()

Testing API endpoint without embedding...

API returned 5 results:

1. 20251124_weekly_kkps_advisory_page_1
   Similarity: 0.6446
   File: 20251124_Weekly_KKPS Advisory.pdf
   Content preview: KKPS ADVISORY | NOVEMBER 2025

CIO VIEW

มุมมองตลาดหุ้นปรับขึ้นดีในปี 2026 จากกำไรที่เติบโตแข็งแรง แ...

2. 2026_kkp_wm_year_ahead_cleared_page_4
   Similarity: 0.6341
   File: 2026_KKP WM Year Ahead_Cleared.pdf
   Content preview: CORE AND SATELLITE PORTFOLIO

logo: CIO VIEW  
Scene Overview :  
  • The image contains a logo cons...

3. 20251124_weekly_kkps_advisory_page_4
   Similarity: 0.6333
   File: 20251124_Weekly_KKPS Advisory.pdf
   Content preview: CORE AND SATELLITE PORTFOLIO

logo:  
  • The image is a logo consisting of a target with an arrow a...

4. 20251124_weekly_kkps_advisory_page_2
   Similarity: 0.6262
   File: 20251124_Weekly_KKPS Advisory.pdf
   Content preview: TOP IDEAS FOR THE WEEK

Summary : This pie chart illustrates the allocation of assets between "Core"...

5. 2025072

## 4. Performance Test

In [ ]:
import time


async def performance_test():
    """Test search performance"""
    test_queries = [
        "portfolio optimization strategies",
        "risk assessment models",
        "asset allocation",
        "market volatility analysis",
        "investment performance metrics",
    ]

    print("Running performance test...")
    print("-" * 60)

    for query in test_queries:
        start_time = time.time()

        results = await test_vector_search(query=query, limit=10, threshold=0.5)

        end_time = time.time()
        duration = (end_time - start_time) * 1000  # Convert to ms

        print(f"Query: '{query}'")
        print(f"Results: {len(results)} found in {duration:.2f}ms")
        print()


await performance_test()

## 5. Summary

This notebook tested:
1. ✅ Database connection and pgvector extension
2. ✅ Basic vector search functionality
3. ✅ Search with file filters
4. ✅ Search with chunk type filters
5. ✅ Multiple filters combined
6. ✅ API endpoint integration (optional)
7. ✅ Performance metrics

The new pgvector implementation successfully replaces Chroma with PostgreSQL while maintaining all search capabilities.